# Histograma

In [25]:
import numba 
from numba import cuda, jit, njit
import numpy as np
import datetime as dt

def now():
    return dt.datetime.now()

size = 10000

with open('offline_1000000.csv', 'r', encoding='utf-8') as arq:
    array = np.array([int(i) for i in arq.read().split('\n') if i])
array.shape, array.dtype

((1000000,), dtype('int64'))

In [22]:
def histograma_np(array, size):
    return np.histogram(array, size)


t0 = now()
hist, bin_edges = histograma_np(array, size)
print(now()-t0)

0:00:00.054000


In [41]:
@numba.jit(nopython=True)
def compute_bin(x, n, xmin, xmax):
    # special case to mirror NumPy behavior for last bin
    if x == xmax:
        return n - 1 # a_max always in last bin
    # SPEEDTIP: Remove the float64 casts if you don't need to exactly reproduce NumPy
    bin = np.int32(n * (np.float64(x) - np.float64(xmin)) / (np.float64(xmax) - np.float64(xmin)))
    if bin < 0 or bin >= n:
        return None
    else:
        return bin
    
@numba.cuda.jit
def histogram_cuda(x, xmin, xmax, histogram_out):
    nbins = histogram_out.shape[0]
    bin_width = (xmax - xmin) / nbins

    start = cuda.grid(1)
    stride = cuda.gridsize(1)

    for i in range(start, x.shape[0], stride):
        # note that calling a numba.jit function from CUDA automatically
        # compiles an equivalent CUDA device function!
        bin_number = compute_bin(x[i], nbins, xmin, xmax)

        if bin_number >= 0 and bin_number < histogram_out.shape[0]:
            numba.cuda.atomic.add(histogram_out, bin_number, 1)

def histograma_cuda(serie, size):
    histogram_out, minimo, maximo = np.zeros(size), np.min(serie), np.max(serie)
    histogram_cuda[64, 64](serie, minimo, maximo, histogram_out)
    return histogram_out

t0 = now()
histogramacu1 = histograma_cuda(array, size)
print(now()-t0)

0:00:01.365001


In [45]:
@numba.jit(nopython=True)
def compute_bin(x, n, xmin, xmax):
    # special case to mirror NumPy behavior for last bin
    if x == xmax:
        return n - 1 # a_max always in last bin
    # SPEEDTIP: Remove the float64 casts if you don't need to exactly reproduce NumPy
    bin = np.int32(n * (np.float64(x) - np.float64(xmin)) / (np.float64(xmax) - np.float64(xmin)))
    if 0 <=  bin < n:
        return bin
    
@numba.cuda.jit
def histogram_cuda(x, xmin, xmax, histogram_out):
    nbins = histogram_out.shape[0]
    bin_width = (xmax - xmin) / nbins

    for i in range(cuda.grid(1), x.shape[0], cuda.gridsize(1)):
        bin_number = compute_bin(x[i], nbins, xmin, xmax)

        if 0 <= bin_number < nbins:
            numba.cuda.atomic.add(histogram_out, bin_number, 1)

def histograma_cuda(serie, size):
    histogram_out, minimo, maximo = np.zeros(size), np.min(serie), np.max(serie)
    histogram_cuda[64, 64](serie, minimo, maximo, histogram_out)
    return histogram_out

t0 = now()
histogramacu2 = histograma_cuda(array, size)
print(now()-t0)

0:00:01.288001


In [44]:
histogramacu1.sum() == histogramacu2.sum()

True